# Deep Learning on MNIST

[Resource](https://numpy.org/numpy-tutorials/content/tutorial-deep-learning-on-mnist.html)

This tutorial demonstrates how to build a simple **feedforward neural network** (with one hidden layer) and train it from scratch with NumPy to recognize handwritten digit images.

# Before We Begin

Feedforward refers to recognition-inference architecture of neural networks. As you already know, artificial neural networks architectures are based on inputs multiplied by weights to obtain outputs.

On the other hand, recurrent neural networks, or neural networks with loops, allow information from later processing stages to feed back to earlier stages for sequence processing. However, at every stage of inference a feedforward multiplication remains the core, essential for backpropagation.

Thus, neural networks cannot contain feedback like negative feedback or positive feedback where the outputs feed back to the very same inputs and modify them, as this will form an infinite loop.

Essentially, it's not possible to rewind in time to generate an error signal through backpropagation.

# Continuing with the lesson

Your deep learning model - one of the most basic artificial neural networks that resembles the original multi-layer perceptron - will learn to classify digits from 0 to 9 from the MNIST dataset.

The dataset contains 60,000 training and 10,000 test images and corresponding labels. Each training and test image is of size 784 (or 28x28 pixels) - this will be your input for the neural network.

Based on the image inputs and their labels (supervised learning), your neural network will be trained to learn their features using forward propagation and backpropagation (reverse-mode differentiation). The final output of the network is a vector of 10 scores - one for each handwritten digit image. You will also be evaluate how good your model is at classifying the images on the test set.

# Load the MNIST Dataset

In this section, you will download the zipped MNIST dataset files. Then, you will transform them into 4 files of NumPy array type using built-in Python modules. Finally, you will split the arrays into training and test sets.

1) Define a variable to store the training/test image/label names of the MNIST dataset in the list:

In [4]:
data_sources = {
    "training_images": "train-images-idx3-ubyte.gz",  # 60,000 training images.
    "test_images": "t10k-images-idx3-ubyte.gz",  # 10,000 test images.
    "training_labels": "train-labels-idx1-ubyte.gz",  # 60,000 training labels.
    "test_labels": "t10k-labels-idx1-ubyte.gz",  # 10,000 test labels.
}

2) Load the data. First check if the data is stored locally; if not, then download it.

In [8]:
import requests
import os

data_dir = "data"
os.makedirs(data_dir, exist_ok=True)

base_url = "https://ossci-datasets.s3.amazonaws.com/mnist/"

for fname in data_sources.values():
    fpath = os.path.join(data_dir, fname)
    if not os.path.exists(fpath):
        print("Downloading file: " + fname)
        resp = requests.get(base_url + fname, stream=True)
        resp.raise_for_status()
        with open(fpath, "wb") as fh:
            for chunk in resp.iter_content(chunk_size=128):
                fh.write(chunk)

3) Decompress the 4 files and create 4 `ndarrays`, saving them into a dictionary. Each original image is of size 28x28 and neural networks normally expect a 1D vector input; therefore, you also need to reshape the images by multiplying 28 by 28 (784).

In [11]:
import gzip
import numpy as np

mnist_dataset = {}

# Images
for key in ("training_images", "test_images"):
    with gzip.open(os.path.join(data_dir, data_sources[key]), "rb") as mnist_file:
        mnist_dataset[key] = np.frombuffer(
            mnist_file.read(), np.uint8, offset=16
        ).reshape(-1, 28 * 28)
# Labels
for key in ("training_labels", "test_labels"):
    with gzip.open(os.path.join(data_dir, data_sources[key]), "rb") as mnist_file:
        mnist_dataset[key] = np.frombuffer(mnist_file.read(), np.uint8, offset=8)

4) Split the data into training and test sets using the standard notation of `x` for data and `y` for labels, calling the training and test set images `x_train` and `x_test`, and the labels `y_train` and `y_test`: